In [1]:
require 'pco_api'

token =  ENV['PCO_API_TOKEN']
secret = ENV['PCO_API_SECRET']

api = PCO::API.new(basic_auth_token: token, basic_auth_secret: secret)

require 'sqlite3'

# Open a database
db = SQLite3::Database.new "songs.db"

nil

In [ ]:


# Create a table
rows = db.execute <<-SQL
  create table songs (
    title varchar(50),
    id varchar(20),
    ccli int,
    themes text

  );
SQL


nil

## Load songs from PCO

In [18]:
db.execute 'delete from songs'

offset = 0
per_page = 25
fetch_more = true

while fetch_more

  response = api.services.v2.songs.get(offset: offset, per_page: per_page)

  data = response["data"]

  data.each do |song|
    attrs = song['attributes']
    title = attrs['title']
    themes = attrs['themes']
    ccli = attrs['ccli_number']
    id = song['id']
    
    db.execute "insert into songs values ( ?, ?, ?, ?)", [ title, id, ccli, themes]
  end
  
  fetch_more = data.length == per_page
  offset = offset + per_page

end

# Find a few rows
db.execute( "select count(*) as count from songs" ) do |row|
  p row[0]
end

nil

79


## Load Arrangements from PCO

In [12]:
db.execute <<-SQL
drop table arrangements;
SQL

# Create table for arrangements
db.execute <<-SQL
  create table arrangements (
    song_id varchar(20),
    id varchar(20),
    name varchar(50),
    bpm int,
    meter varchar(10),
    notes text,
    length int,
    sequence text,
    has_chords varchar(10),
    has_chor_chart varchar(10)
  );
SQL

nil

In [13]:
# Load arrangements from PCO
db.execute( "select id from songs" ) do |row|
  song_id = row[0]
  offset = 0
  per_page = 100

  response = api.services.v2.songs[song_id.to_i].arrangements.get(offset: offset, per_page: per_page)

  data = response["data"]

  data.each do |arrangement|
    id = arrangement['id']
    attrs = arrangement['attributes']

    bpm = attrs["bpm"]
    has_chord_chart = attrs['has_chord_chart']
    has_chords = attrs["has_chords"]
    length = attrs['length']
    meter = attrs['meter']
    name = attrs['name']
    notes = attrs['notes']
    sequence = attrs['sequence']
    
    db.execute "insert into arrangements values ( ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)",
      [song_id, id, name, bpm, meter, notes, length, sequence.to_json, has_chords.to_s, has_chord_chart.to_s]
  end
end


nil

In [14]:
# Find a few rows
db.execute( "select count(*) as count from arrangements" ) do |row|
  p row[0]
end

nil

84


## Load Schedule for songs from PCO
(This actually isn't helpful because it's only the future schedule, not historical)

In [23]:
# db.execute <<-SQL
# drop table song_schedules;
# SQL

# Create table for arrangements
db.execute <<-SQL
  create table song_schedules (
    id varchar(20),
    song_id varchar(20),
    arrangement_id varchar(20),
    key_id varchar(20),
    plan_id varchar(20),
    service_type_id varchar(20),
    item_id varchar(20)
  );
SQL

nil

In [31]:
db.execute( "delete from song_schedules" )

# Load arrangements from PCO
db.execute( "select id from songs" ) do |row|
  
  song_id = row[0]
  offset = 0
  per_page = 100
  fetch_more = true

  while fetch_more

    response = api.services.v2.songs[song_id.to_i].song_schedules.get(offset: offset, per_page: per_page)

    data = response["data"]

    data.each do |song_schedule|
      id = song_schedule['id']
      attrs = song_schedule['attributes']
      rel = song_schedule['relationships']

      arrangement_id = rel['arrangement']['data']['id']
      key_id = rel['key']['data']['id']
      plan_id = rel['plan']['data']['id']
      service_type_id = rel['service_type']['data']['id']
      item_id = rel['item']['data']['id']
      
      db.execute "insert into song_schedules values ( ?, ?, ?, ?, ?, ?, ?)",
        [id, song_id, arrangement_id, key_id, plan_id, service_type_id, item_id]
    end
    
    fetch_more = data.length == per_page
    offset = offset + per_page
  end
end


nil

In [29]:
# Find a few rows
db.execute( "select * from song_schedules" ) do |row|
  p row
end

nil

["482937461", "13327207", "14986695", "19063666", "35443202", "643436", "482937461"]
["482937466", "13581453", "15275263", "19192222", "35443202", "643436", "482937466"]
["482937457", "13659297", "15363891", "19285176", "35443202", "643436", "482937457"]
["482937460", "13774675", "15496780", "19499501", "35443202", "643436", "482937460"]


## Load Service Types

In [35]:
# db.execute <<-SQL
# drop table song_schedules;
# SQL

# Create table for arrangements
db.execute <<-SQL
  create table service_types (
    id varchar(20),
    name varchar(50)
  );
SQL

nil

In [36]:
  offset = 0
  per_page = 100
  fetch_more = true

  while fetch_more

    response = api.services.v2.service_types.get(offset: offset, per_page: per_page)

    data = response["data"]

    data.each do |service_type|
      id = service_type['id']
      attrs = service_type['attributes']
      
      name = attrs['name']
      
      db.execute "insert into service_types values ( ?, ?)", [id, name]
    end
    
    fetch_more = data.length == per_page
    offset = offset + per_page
  end

In [37]:
# Find a few rows
db.execute( "select * from service_types" ) do |row|
  p row
end

nil

["643436", "TCBR Sunday Morning Service"]
["645713", "TCBR Kids"]
["670741", "Test Plan"]
["671932", "Family Mtg."]


## Load Plans

In [44]:
# db.execute <<-SQL
# drop table song_schedules;
# SQL

# Create table for arrangements
db.execute <<-SQL
  create table plans (
    id varchar(20),
    service_type_id varchar(20),
    dates text
  );
SQL

nil

SQLite3::SQLException: table plans already exists

In [72]:
require 'date'
db.execute( "delete from plans" )

# Load service types from PCO
db.execute( "select id from service_types" ) do |row|
  
  service_type_id = row[0]
  offset = 0
  per_page = 100
  fetch_more = true

  while fetch_more

    response = api.services.v2.service_types[service_type_id.to_i].plans.get(offset: offset, per_page: per_page)

    data = response["data"]

    data.each do |plan|
      id = plan['id']
      attrs = plan['attributes']
      rel = plan['relationships']
        
      dates = attrs['dates'] == 'No dates' ? nil : Date.strptime(attrs['dates'], '%B %d, %Y')
      short_dates = attrs['short_dates']
      
      db.execute "insert into plans values ( ?, ?, ?)", [id, service_type_id, dates.to_s]
    end
    
    fetch_more = data.length == per_page
    offset = offset + per_page
  end
end


nil

"October 9, 2016"
"October 16, 2016"
"October 23, 2016"
"October 30, 2016"
"November 6, 2016"
"November 13, 2016"
"November 20, 2016"
"November 27, 2016"
"December 4, 2016"
"December 11, 2016"
"December 18, 2016"
"January 15, 2017"
"January 22, 2017"
"January 29, 2017"
"February 5, 2017"
"No dates"
"February 19, 2017"
"March 5, 2017"
"March 12, 2017"
"March 19, 2017"
"March 26, 2017"
"February 12, 2017"
"February 26, 2017"
"April 2, 2017"
"April 9, 2017"
"April 16, 2017"
"April 23, 2017"
"April 30, 2017"
"May 7, 2017"
"May 14, 2017"
"May 21, 2017"
"May 28, 2017"
"June 4, 2017"
"June 11, 2017"
"June 18, 2017"
"June 25, 2017"
"July 2, 2017"
"July 9, 2017"
"July 16, 2017"
"July 23, 2017"
"July 30, 2017"
"August 6, 2017"
"August 13, 2017"
"August 20, 2017"
"August 27, 2017"
"September 3, 2017"
"September 10, 2017"
"September 17, 2017"
"September 24, 2017"
"October 1, 2017"
"October 8, 2017"
"October 15, 2017"
"October 22, 2017"
"October 29, 2017"
"November 5, 2017"
"November 12, 2017"
"Nov

In [46]:
# Find a few rows
db.execute( "select * from plans" ) do |row|
  p row
end

nil

["28375873", "643436", "October 9, 2016"]
["28387693", "643436", "October 16, 2016"]
["28435269", "643436", "October 23, 2016"]
["28501454", "643436", "October 30, 2016"]
["28614855", "643436", "November 6, 2016"]
["28614986", "643436", "November 13, 2016"]
["28615025", "643436", "November 20, 2016"]
["28831218", "643436", "November 27, 2016"]
["28831226", "643436", "December 4, 2016"]
["28831229", "643436", "December 11, 2016"]
["29043169", "643436", "December 18, 2016"]
["29158007", "643436", "January 15, 2017"]
["29158011", "643436", "January 22, 2017"]
["29158012", "643436", "January 29, 2017"]
["29158016", "643436", "February 5, 2017"]
["29158031", "643436", "No dates"]
["29180606", "643436", "February 19, 2017"]
["29885238", "643436", "March 5, 2017"]
["29885465", "643436", "March 12, 2017"]
["29885466", "643436", "March 19, 2017"]
["29885564", "643436", "March 26, 2017"]
["29912297", "643436", "February 12, 2017"]
["30069323", "643436", "February 26, 2017"]
["30270721", "643436"

## Plan Items

In [56]:
# db.execute <<-SQL
# drop table song_schedules;
# SQL

# Create table for arrangements
db.execute <<-SQL
  create table plan_items (
    id varchar(20),
    plan_id varchar(20),
    title text,
    description text,
    item_type text,
    length int,
    service_position text,
    service_sequence int,
    custom_arrangement_sequence text,
    song_id varchar(20),
    arrangement_id varchar(20),
    key_id varchar(20)
  );
SQL

nil

In [58]:
#db.execute( "delete from plan_items" )

# Load service types from PCO
db.execute( "select id, service_type_id from plans" ) do |row|
  plan_id = row[0]
  service_type_id = row[1]
  offset = 0
  per_page = 100
  fetch_more = true

  while fetch_more

    response = api.services.v2.service_types[service_type_id.to_i].plans[plan_id.to_i].items.get(offset: offset, per_page: per_page)

    data = response["data"]

    data.each do |plan|
      id = plan['id']
      attrs = plan['attributes']
      rel = plan['relationships']
      
      custom_arrangement_sequence = attrs['custom_arrangement_sequence']
      description = attrs['description']
      item_type = attrs['item_type']
      length = attrs['length']
      sequence = attrs['sequence']
      service_position = attrs['service_position']
      title = attrs['title']
      
      song_id = rel.dig('song', 'data', 'id')
      arrangement_id = rel.dig('arrangement', 'data', 'id')
      key_id = rel.dig('key', 'data', 'id')
            
      values = [id, plan_id, title, description, item_type, length, service_position, sequence, custom_arrangement_sequence.to_json, song_id, arrangement_id, key_id]
      db.execute "insert into plan_items values ( ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)", values
    end
    
    fetch_more = data.length == per_page
    offset = offset + per_page
  end
end


nil

In [60]:
# Find a few rows
db.execute( "select count(*) from plan_items" ) do |row|
  p row
end

nil

[3755]


## Plot frequency diagram

In [74]:
db.execute(<<-SQL
  SELECT
    s.title,
    p.dates
  FROM
    plan_items pi
    JOIN songs s ON s.id = pi.song_id
    JOIN plans p on p.id = pi.plan_id
  ORDER BY
    p.dates desc
SQL
  ) do |row|
    p row
  end

nil

["How Rich A Treasure We Possess", "2018-08-26"]
["Hail The Day That Christ Arose", "2018-08-26"]
["Hallelujah What A Savior (Your Love Has Rescued Me)", "2018-08-26"]
["Stronger", "2018-08-26"]
["This Is Amazing Grace", "2018-08-19"]
["Come Thou Fount Of Every Blessing", "2018-08-19"]
["Come Behold The Wondrous Mystery", "2018-08-19"]
["Jesus Is Better", "2018-08-19"]
["Oh For A Thousand Tongues To Sing", "2018-08-12"]
["How Firm A Foundation", "2018-08-12"]
["Hallelujah What A Savior (Your Love Has Rescued Me)", "2018-08-12"]
["All Glory Be To Christ", "2018-08-12"]
["Grace Alone", "2018-08-05"]
["Hail The Day That Christ Arose", "2018-08-05"]
["Come Behold The Wondrous Mystery", "2018-08-05"]
["How Rich A Treasure We Possess", "2018-08-05"]
["Oh For A Thousand Tongues To Sing", "2018-07-29"]
["Christ Is Enough", "2018-07-29"]
["Sovereign", "2018-07-29"]
["Jesus Is Better", "2018-07-29"]
["There Is One Reason", "2018-07-22"]
["Man Of Sorrows", "2018-07-22"]
["Come Behold The Wondrous

["The Solid Rock", "2016-11-20"]
["This Is Amazing Grace", "2016-11-20"]
["In Christ Alone", "2016-11-20"]
["How Deep The Father's Love For Us", "2016-11-20"]
["Grace Alone", "2016-11-20"]
["How Great Is Our God", "2016-11-13"]
["This Is Amazing Grace", "2016-11-13"]
["In Christ Alone", "2016-11-13"]
["Cornerstone", "2016-11-13"]
["Jesus Thank You", "2016-11-13"]
["This Is Amazing Grace", "2016-11-06"]
["Come Thou Fount", "2016-11-06"]
["Christ Be All Around Me", "2016-11-06"]
["Jesus Paid It All", "2016-11-06"]
["Be Thou My Vision", "2016-11-06"]
["Come Lord Jesus (Even So Come)", "2016-10-30"]
["Christ Is Risen", "2016-10-30"]
["This I Believe", "2016-10-30"]
["Sovereign Over Us", "2016-10-30"]
["Always", "2016-10-30"]
["Come Thou Fount", "2016-10-23"]
["Man Of Sorrows", "2016-10-23"]
["Before The Throne Of God Above", "2016-10-23"]
["Be Thou My Vision", "2016-10-23"]
["O Great God", "2016-10-23"]
["10,000 Reasons (Bless The Lord)", "2016-10-16"]
["Jesus Thank You", "2016-10-16"]
["I